In [1]:
import os
import numpy

spark_home = os.environ.get('SPARK_HOME', None) 

In [82]:
from pyspark.sql import DataFrame
from pyspark.ml import Pipeline
from pyspark.ml.classification import *
from pyspark.ml.feature import *
from pyspark.ml.evaluation import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.clustering import *

In [100]:
input_data = spark.read.csv("/user/hduser1/flattened.csv", header=True,inferSchema=True).repartition(100)

In [45]:
input_data.head()

Row(_c0=3556163, red=94.0, green=160.0, blue=152.0)

In [101]:
# recreate the index
input_data.createOrReplaceTempView("input_data")
x=spark.sql("SELECT _c0,_c0 DIV (512*512) AS frame, (_c0 DIV 512)%(512) AS x, _c0%(512) AS y FROM input_data")

In [102]:
x.createOrReplaceTempView('x')
input_data = spark.sql("SELECT x._c0,frame, x, y, red, green, blue FROM input_data, x WHERE input_data._c0 = x._c0")

In [103]:
ignore = ['_c0','frame','x','y']
assembler = VectorAssembler(
    inputCols=[x for x in input_data.columns if x not in ignore],
    outputCol='features')

input_data= assembler.transform(input_data)


In [125]:
bkm = BisectingKMeans(k=8, minDivisibleClusterSize=1.0)
bkmmodel = bkm.fit(input_data)
result = bkmmodel.transform(input_data)

In [145]:
result = result.orderBy('_c0').drop('features')
result.write.csv('hdfs://master:9000/user/hduser1/results.csv')

AnalysisException: u'path hdfs://master:9000/user/hduser1/results.csv already exists.;'

In [144]:
result.write.csv(os.path.join(tempfile.mkdtemp(), 'data'))

NameError: name 'tempfile' is not defined